In [1]:
import os
from datetime import datetime
import json
import numpy as np
import matplotlib.pyplot as plt
# from osgeo import gdal
import iris
import iris.plot as iplt
import iris.quickplot as qplt
import iris_grib
import iris.util
import iris.pandas
from iris.analysis import trajectory 
from iris.coords import DimCoord
from iris.cube import Cube
# from iris.experimental.raster import export_geotiff
import pandas as pd
import openpyxl
import iris.coord_categorisation

In [2]:
wstations_file = pd.read_csv(r"./Meteo_Stations.csv", sep=";")
wstations_df = pd.DataFrame(wstations_file)

print (wstations_df)

ID_sel = input("Enter ID of Station to extract elevation or type 'c' for custom elevation: ")
if ID_sel == "c":
    w_elevation = int(input("Enter custom elevation in meters: "))
    print(w_elevation)
else:    
    w_elevation = wstations_df.loc[int(ID_sel),"Elevation"]
    print("Selected Elevation: " + str(w_elevation))

ELR = input("Set ELR as [T/m]: ") ## Usually 0.005

                     Name  Elevation        Lat        Lon
0                 Agerola        848  40,646944  14,541111
1           Ariano Irpino        631  41,196944  15,136111
2               Benevento        236  41,115000  14,825000
3                 Cellole          9  41,196111  13,838056
4    Conza della Campania        770  40,861944  15,281944
5             Lago Patria          1  40,941944  14,185833
6                Montella        515  40,838056  15,038889
7    Montesano Marcellana        552  40,256111  15,663889
8                  Nisida         88  40,793889  14,163889
9             Postiglione        660  40,561944  15,236944
10        Rocca d'Evandro         62  41,425000  13,880000
11          S. Bartolomeo        750  41,421944  15,041111
12  San Marco Evangelista         31  41,020000  14,343889
13  S. Salvatore Telesino        167  41,246944  14,473056
14                Salerno         13  40,643889  14,836944
15           Torre Orsaia        413  40,131944  15,4588

In [3]:
print(ELR, w_elevation)

0.005 110


In [4]:
""" Calculate new temperature from GRIB temperature, 
GRIB elevation and temperature variation along elevation equation """


geopotential = "geopotential.grib2"
geopot_cube = iris.load_cube(geopotential)

# Geopotential to elevation and conversion in meters
elev_cube = geopot_cube / 9.80665
elev_cube.rename("Elevation")
elev_cube.units = "m"

print(elev_cube)

Elevation / (m)                     (latitude: 1801; longitude: 3600)
    Dimension coordinates:
        latitude                             x                -
        longitude                            -                x
    Scalar coordinates:
        forecast_period             0 hours
        forecast_reference_time     2013-08-09 12:00:00
        time                        2013-08-09 12:00:00
    Attributes:
        GRIB_PARAM                  GRIB2:d000c003n004
        centre                      'European Centre for Medium Range Weather Forecasts'


In [6]:
#     crop_elevation()
with open('area.txt', 'r') as filehandle:
    area = json.load(filehandle)
    area[1] = area[1] - 0.10

area_slicer = iris.Constraint(longitude=lambda v: area[1] <= v <= area[3], latitude=lambda v: area[2] <= v <= area[0])
elevcube_slice = elev_cube.extract(area_slicer)

print(elevcube_slice)

Elevation / (m)                     (latitude: 18; longitude: 27)
    Dimension coordinates:
        latitude                             x              -
        longitude                            -              x
    Scalar coordinates:
        forecast_period             0 hours
        forecast_reference_time     2013-08-09 12:00:00
        time                        2013-08-09 12:00:00
    Attributes:
        GRIB_PARAM                  GRIB2:d000c003n004
        centre                      'European Centre for Medium Range Weather Forecasts'


In [8]:
#     load_ERA5_cube()    
cube = "ET0_t2m_july.grib"
air_temperature = iris.load_cube(cube)
air_temperature.convert_units('celsius')

print(air_temperature)

air_temperature / (celsius)         (time: 744; latitude: 20; longitude: 28)
    Dimension coordinates:
        time                             x              -              -
        latitude                         -              x              -
        longitude                        -              -              x
    Auxiliary coordinates:
        forecast_period                  x              -              -
    Scalar coordinates:
        height                      2 m
        originating_centre          European Centre for Medium Range Weather Forecasts


In [12]:
# New temperature calculation via elevation temperature change equation
t_shift = (w_elevation - elevcube_slice) * float(ELR) ## = (weather_station_elevation - elevetion_cube) * ELR [T/m]
t_shift.rename("shift_temperature")
t_shift.units = "celsius"

# # Check if cubes coordinates points are the same and replace axis
# for axis in ('x', 'y'):
#     dimcoord_1 = air_temperature.coord(axis=axis, dim_coords=True)
#     dimcoord_2 = t_shift.coord(axis=axis, dim_coords=True)
# # Ensure they are basically the same, before proceeding.
#     assert np.allclose(dimcoord_1.points, dimcoord_2.points)
#     assert dimcoord_1.has_bounds() == dimcoord_2.has_bounds()
# if dimcoord_1.has_bounds():
#     assert np.allclose(dimcoord_1.bounds, dimcoord_2.bounds)
# # change cube#2 to use coords from cube#1 
# t_shift.replace_coord(dimcoord_1)

# # Replace coordinates anyway
# t_shift.replace_coord(air_temperature.coord(axis="X"))
# t_shift.replace_coord(air_temperature.coord(axis="Y"))

# Do main calculation
# t_new = air_temperature[:] + t_shift
# t_new.rename("interpolated_temperature")

print(air_temperature.coord("latitude").points, t_shift.coord("latitude").points, elevcube_slice.coord("latitude").points)

[41.65 41.55 41.45 41.35 41.25 41.15 41.05 40.95 40.85 40.75 40.65 40.55
 40.45 40.35 40.25 40.15 40.05 39.95 39.85 39.75] [41.6 41.5 41.4 41.3 41.2 41.1 41.  40.9 40.8 40.7 40.6 40.5 40.4 40.3
 40.2 40.1 40.  39.9] [41.6 41.5 41.4 41.3 41.2 41.1 41.  40.9 40.8 40.7 40.6 40.5 40.4 40.3
 40.2 40.1 40.  39.9]


In [ ]:
#     interpolate_atpoint()
new_lat = input("Set Latitude of interpolation as decimal: ")
new_lon = input("Set Longitude of interpolation as decimal: ")
    
sample_points = [('latitude', new_lat), ('longitude', new_lon)]
t_new_interpolated = t_new.interpolate(sample_points, iris.analysis.Nearest(extrapolation_mode='extrapolate'))

In [ ]:
#  plot_timeseries()
""" Plot interpolated at point time series """
#fname = iris.sample_data_path("air_temp.pp")
#temperature = iris.load_cube(fname)

# Take a 1d slice using array style indexing.
#temperature_1d = temperature[5, :]

qplt.plot(t_new_interpolated)

plt.show()

In [ ]:
#  save_toGRIB() # Optional - to use with rioxarray
iris_grib.save_grib2(t_new, "t_new.grib2") # Check how to set 9999 to masked data (or no data)



In [11]:
print ('The script took {0} seconds !'.format(datetime.now() - startTime))

In [8]:
# def geopot_to_elev():
#     """ Geopotential to elevation and conversion in meters """
#     global elev_cube

#     # Load Geopotential cube
#     geopotential = "geopotential.grib2"
#     geopot_cube = iris.load_cube(geopotential)

#     # Geopotential to elevation and conversion in meters
#     elev_cube = geopot_cube / 9.80665
#     elev_cube.rename("Elevation")
#     elev_cube.units = "m"

#     return elev_cube

In [9]:
# def crop_elevation():
#     """ Crop elevetion cube to sample area coordinates taken from grib_download CSV exported file """
#     global elevcube_slice

#     with open('area.txt', 'r') as filehandle:
#         area = json.load(filehandle)
#         area[1] = area[1] - 0.1

#     area_slicer = iris.Constraint(longitude=lambda v: area[1] <= v <= area[3], latitude=lambda v: area[2] <= v <= area[0])
#     elevcube_slice = elev_cube.extract(area_slicer)

#     return elevcube_slice

In [10]:
# def load_ERA5_cube():
#     """ Load downloaded cube from ERA5 (cdsapi API) """
#     global air_temperature
    
#     cube = "ET0_t2m_july.grib"
#     air_temperature = iris.load_cube(cube)
#     air_temperature.convert_units('celsius')

#     return air_temperature

In [ ]:
# def interpolate_atpoint():
#     """ Interpolate at given point coordinates """
#     global t_new_interpolated
#     new_lat = input("Set Latitude of interpolation as decimal: ")
#     new_lon = input("Set Longitude of interpolation as decimal: ")
    
#     sample_points = [('latitude', new_lat), ('longitude', new_lon)]
#     t_new_interpolated = t_new.interpolate(sample_points, iris.analysis.Nearest(extrapolation_mode='extrapolate'))

#     return t_new_interpolated

In [ ]:
# def save_toNetCDF():
#     iris.save(t_new, "t_new.nc")
    
#     return

In [12]:
# def save_toGRIB():
#     # export_geotiff(t_new[1], "t_new.tif")
    
#     iris_grib.save_grib2(t_new, "t_new.grib2") # Check how to set 9999 to masked data (or no data)

#     return

In [ ]:
# def save_toxls():
#     """ Save results to Pandas Data Frame and export to Excel file """   
    
#     ## 1D cube to panda series
#     t_new_pandas = iris.pandas.as_series(t_new_interpolated, copy=True)
#     t_new_pandas.to_excel(r"t_new_interpolated.xlsx", index = True, header=True)
    
#     ## For 2D cubes
#     # t_new_pandas = iris.pandas.as_data_frame(t_new_interpolated[:], copy=True)
#     # t_new_pandas.to_excel(r"t_new.xlsx", index = True, header=True)
    
#     return

In [ ]:
# def save_tocsv():
#     """ Save results to Pandas Data Frame and export to CSV file """
    
#     ## 1D cube to panda series
#     t_new_pandas = iris.pandas.as_series(t_new_interpolated, copy=True)
#     t_new_pandas.to_csv("t_new_interpolated.csv")
    
#     ## For 2D cubes
#     # t_new_pandas = iris.pandas.as_data_frame(t_new, copy=True)
#     # t_new_pandas.to_csv("t_new.csv")
    
#     return

In [ ]:
# def plot_timeseries():
#     """ Plot interpolated at point time series """
#     #fname = iris.sample_data_path("air_temp.pp")
#     #temperature = iris.load_cube(fname)

#     # Take a 1d slice using array style indexing.
#     #temperature_1d = temperature[5, :]

#     qplt.plot(t_new_interpolated)

#     plt.show()

In [ ]:
# if __name__ == "__main__":
#     main()